<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/a/a8/%D0%9B%D0%9E%D0%93%D0%9E_%D0%A8%D0%90%D0%94.png" width=500px/>
    <font>Python 2020</font><br/>
    <br/>
    <br/>
    <b style="font-size: 2em">Простые паттерны проектирования,</b><br/>
    <b style="font-size: 2em">метаклассы, дескрипторы</b><br/>
    <br/>
    <font>Михаил Максимов</font><br/>
</center>

In [3]:
from abc import abstractmethod, ABC
import typing as tp

TRow = tp.Dict[str, tp.Any]
TRowsIterable = tp.Iterable[TRow]
TRowsGenerator = tp.Generator[TRow, None, None]

class Operation(ABC):
    @abstractmethod
    def __call__(self, rows: TRowsIterable, *args: tp.Any, **kwargs: tp.Any) -> TRowsGenerator:
        pass

### ABC

In [ ]:
class Bird():
    def fly(self):
        raise NotImplementedError()
        
class Crow(Bird):
    pass

print(1)
c = Crow()
print(2)
c.fly()
print(3)

In [2]:
class Bird():
    def fly(self):
        raise NotImplementedError()
        
class Crow(Bird):
    pass

print(1)
c = Crow()
print(2)
c.fly()
print(3)

1
2


NotImplementedError: 

In [ ]:
from abc import ABC, abstractmethod

class Bird(ABC):
    @abstractmethod
    def fly(self):
        pass

class Crow(Bird):
    pass

print(1)
c = Crow()
print(2)
c.fly()
print(3)

In [3]:
from abc import ABC, abstractmethod

class Bird(ABC):
    @abstractmethod
    def fly(self):
        pass

class Crow(Bird):
    pass

print(1)
c = Crow()
print(2)
c.fly()
print(3)

1


TypeError: Can't instantiate abstract class Crow with abstract methods fly

In [4]:
from abc import ABC, abstractmethod

class Iterable(ABC):
    @abstractmethod
    def __iter__(self) -> tp.Any:
        pass

class Sized(ABC):
    @abstractmethod
    def __len__(self) -> int:
        pass

class Container(ABC):
    @abstractmethod
    def __contains__(self, x: tp.Any) -> bool:
        pass
    
class Collection(Sized, Iterable, Container):
    pass

### А также

In [5]:
class Reducer(ABC):
    """Base class for reducers"""
    @abstractmethod
    def __call__(self, group_key: tp.Tuple[str, ...], rows: TRowsIterable) -> TRowsGenerator:
        """
        :param rows: table rows
        """
        pass


class Reduce(Operation):
    def __init__(self, reducer: Reducer, keys: tp.Sequence[str]) -> None:
        self.reducer = reducer
        self.keys = keys

    def __call__(self, rows: TRowsIterable, *args: tp.Any, **kwargs: tp.Any) -> TRowsGenerator:
        pass

![alt text](bridge.png "Bridge")

### Bridge
https://sourcemaking.com/design_patterns/bridge

Паттерн проектирования - обобщенное и воспроизводимое решение распространенной проблемы в области инженерии ПО

![alt text](factory.png "Factory")

https://sourcemaking.com/design_patterns/bridge

![no_bridge](no_bridge.png "No bridge")

In [8]:
class FirstRecordReducer(Reducer):
    def __call__(self, group_key: tp.Tuple[str, ...], rows: TRowsIterable) -> TRowsGenerator:
        yield next(iter(rows))

In [9]:
def test_first_record_reducer_returns_first_record():
    input_data = [
        {"a": 1, "b": 1},
        {"a": 1, "b": 2}
    ]
    reducer = FirstRecordReducer()
    result = list(reducer("a", input_data))
    assert result[0] == input_data[0]
    
test_first_record_reducer_returns_first_record()

In [10]:
import itertools
import operator
        
class SortReduce(Reduce):
    def __call__(self, rows: TRowsIterable, *args: tp.Any, **kwargs: tp.Any) -> TRowsGenerator:
        data = sorted(rows, key=operator.itemgetter(*self.keys))# better use Sort
        for key, group in itertools.groupby(data, key=operator.itemgetter(*self.keys)):
            yield self.reducer(key, list(group)) 

In [11]:
class GroupKeyReducer(Reducer):        
    def __call__(self, group_key: tp.Tuple[str, ...], rows: TRowsIterable) -> TRowsGenerator:
        return group_key
        

def test_sort_reduce():
    input_data = [
        {"a": 1},
        {"a": 1},
        {"a": 2},
        {"a": 2},
        {"a": 1},
    ]
    reduce = SortReduce(reducer=GroupKeyReducer(), keys=("a", ))
    result = list(reduce(input_data))
    assert len(result) == len(set(result)) 
    
test_sort_reduce()

## Лекция про два слова

![maintainability](maintainability.png "Maintainability")

![extendability](extendability.png "EXtensibility")

1. Три распространённые проблемы
2. Три паттерна проектирования
3. Дескрипторы
4. Property
5. Путь создания объекта (Метаклассы)
6. Свой ABC

### Проблема 1

In [12]:
Server = tp.TypeVar

In [ ]:
def send_data(data: str, recipient: Server) -> None:
    if data is not None:
        if len(data) > 0 and not data.isspace():
            recipient.process(data)
        else:
            raise ValueError("Data is empty")
    else:
        raise ValueError("Data is invalid")

In [ ]:
def receive_data(sender: Server) -> str:
    data = sender.recieve()
    if data is not None:
        if len(data) > 0 and not data.isspace():
            return data
        else:
            raise ValueError("Data is empty")
    else:
        raise ValueError("Data is invalid")

### Copy-paste base development
https://sourcemaking.com/antipatterns/cut-and-paste-programming

In [ ]:
def validate_data(data: str) -> None:
    if data is None:
        raise ValueError("Data is invalid")
    elif len(data) == 0 or data.isspace():
        raise ValueError("Data is empty")
    
def send_data(data: str, recipient: Server) -> None:
    validate_data(data)
    recipient.process(data)

def receive_data(sender: Server) -> str:
    data = sender.recieve()
    validate_data(data)
    return data

### Проблема 2

In [ ]:
def is_less_than_zero(x: float) -> bool:
    return x < 0

def is_greater_than_one(x: float) -> bool:
    return x > 1

def weird_modulo(x: float) -> float:
    if is_less_than_zero(x):
        return -x
    elif is_greater_than_one(x):
        return 0
    else:
        return x

### Too abstract development

In [ ]:
def weird_modulo(x: float) -> float:
    if x < 0:
        return -x
    elif x > 1:
        return 0
    else:
        return x

### Проблема 3

In [ ]:
class Bird(ABC):
    @abstractmethod
    def eat(self) -> None:
        pass

    @abstractmethod
    def fly(self) -> None:
        pass

In [ ]:
class Crow(Bird):
    def eat(self) -> None:
        # some code
        
    def fly(self) -> None:
        # some code

In [ ]:
birds = [...]
for bird in birds:
    bird.eat()
    bird.fly()

Угадайте какая птица на следующем слайде?

## Какапо
![kakapo](kakapo.jpg "Kakapo")

In [ ]:
class Kakapo(Bird):
    def eat(self) -> None:
        # some code

    def fly(self) -> None:
        # ???

## Scott Meyers
## Effective C++: 55 Specific Ways to Improve Your Programs and Designs
### Item 32: Make sure public inheritance models "is-a"

In [ ]:
# Решение 1
class Kakapo(Bird):
    def eat(self) -> None:
        # some code

    def fly(self) -> None:
        raise NotImplementedError()  # Какапо умеет летать, но это вызывает ошибку

In [ ]:
for bird in birds:
    bird.fly()

In [ ]:
# Решение 2

class Kakapo(Bird):
    def eat(self):
        # some code

    def fly(self):
        pass  # Какапо умеет летать, но в полёте ничего не происходит

## - Из-за чего у нас проблема?

## - В нашей модели какапо не является птицей!

# <font color=red> Наследование всегда выражает зависимость "есть" </font>

In [ ]:
class Parent:
    pass

class Child(Parent):
    pass

# Child есть Parent

# Принцип подстановки Барбары Лисков
## Подкласс не должен требовать от вызывающего кода больше, чем базовый класс.
## И не должен предоставлять вызывающему коду меньше, чем базовый класс.

In [ ]:
# Решение 3

class Bird(ABC):
    @abstractmethod
    def eat(self):
        pass

class FlyingBird(Bird):
    @abstractmethod
    def fly(self):
        pass
    
class Crow(FlyingBird):
    def eat(self) -> None:
        # some code
        
    def fly(self) -> None:
        # some code
    
class Kakapo(Bird):
    def eat(self) -> None:
        # some code

In [ ]:
# Решение 4

class Beak(ABC):  # клюв
    @abstractmethod
    def eat(self):
        pass
    
class SharpBeak(Beak):  # острый клюв
    # some code
    
class ShortBeak(Beak):  # короткий клюв
    # some code

In [ ]:
class Wings(ABC):  # крылья
    @abstractmethod
    def fly(self):
        pass
    
class FlyingWings(Wings):  # летучие крылья
    # some code
    
class FlylessWings(Wings):  # нелетучие крылья
    # some code

In [ ]:
class Bird:
    def __init__(self, wings: Wings, beak: Beak) -> None:
        self.wings = wings
        self.beak = beak
        
    def eat(self) -> None:
        self.beak.eat()
        
    def fly(self) -> None:
        self.wings.fly()
        
class Crow(Bird):
    def __init__(self) -> None:
        super().__init__(FlyingWings(), SharpBeak())
        
class Kakapo(Bird):
    def __init__(self) -> None:
        super().__init__(FlylessWings(), ShortBeak())

# <font color=green> Наследование всегда выражает зависимость "есть" </font>

# <font color=blue> Наследование всегда выражает зависимость "есть" </font>

# <font color=red> Наследование всегда выражает зависимость "есть" </font>

## Паттерны
### Паттерн 1

In [22]:
# /system/unix.py
class NativeUnixButton(object):
    def set_color(self, color: str) -> None:
        pass

class NativeUnixWindow(object):
    def add_button(self, button: NativeUnixButton) -> None:
        pass

In [23]:
# /system/windows.py
class NativeWindowsButton(object):
    def __init__(self) -> None:
        self.color = None

class NativeWindowsWindow(object):
    def addButton(self, button: NativeWindowsButton) -> None:
        pass

1. Создать кнопку
2. Покрасить кнопку
3. Создать окно
4. Добавить кнопку на окно

In [24]:
# /project/main.py

import platform

if platform.system() in ("Unix", "Darwin"):
    window = NativeUnixWindow()
    button = NativeUnixButton()
    button.set_color("red")
    window.add_button(button)
elif platform.system() == "Windows":
    window = NativeWindowsWindow()
    button = NativeWindowsButton()
    button.color = (255, 0, 0)
    window.addButton(button)
else:
    raise ValueError("Unsupported platform")

# Adapter
https://sourcemaking.com/design_patterns/adapter

![no_adapter](no_adapter.png "No adapter")

![adapter](adapter.png "Adapter")

In [25]:
# /project/interface.py
# Создаём общий интерфейс взаимодействия

from abc import ABC, abstractmethod
from enum import Enum


class Color(Enum):
    RED = 1
    GREEN = 2


class ButtonAdapter(ABC):
    @abstractmethod
    def set_color(self, color: Color) -> None:
        pass
    
    
class WindowAdapter(ABC):
    @abstractmethod
    def add_button(self, button: ButtonAdapter) -> None:
        pass

In [26]:
# /project/unix.py
# Реализуем интерфйес для unix

def _color_to_str(color: Color):
    if color == Color.RED:
        return "red"
    elif color == Color.GREEN:
        return "green"
    else:
        raise ValueError("Invalid color for unix: '{}'".format(color))


class UnixButton(ButtonAdapter):
    def __init__(self) -> None:
        self._native_button = NativeUnixButton()

    def set_color(self, color: Color) -> None:
        self._native_button.set_color(_color_to_str(color))
        
        
class UnixWindow(WindowAdapter):
    def __init__(self) -> None:
        self._native_window = NativeUnixWindow()
        
    def add_button(self, button: ButtonAdapter) -> None:
        assert isinstance(button, UnixButton)  # may be done with visitor
        self._native_window.add_button(button._native_button)

In [27]:
# /project/windows.py
# Реализуем интерфйес для windows

def _color_to_tuple(color: Color) -> tp.Tuple[float, float, float]:
    if color == Color.RED:
        return (255, 0, 0)
    elif color == Color.GREEN:
        return (0, 255, 0)
    else:
        raise ValueError("Invalid color for unix: '{}'".format(color))


class WindowsButton(ButtonAdapter):
    def __init__(self) -> None:
        self._native_button = NativeWindowsButton()
    
    def set_color(self, color: Color) -> None:
        self.native_button.color = _color_to_tuple(color)
        
        
class WindowsWindow(WindowAdapter):
    def add_button(self, button: ButtonAdapter) -> None:
        assert isinstance(button, WindowsButton)  # may be done with visitor
        self.native_window.addButton(button.native_button)

In [28]:
if platform.system() in ("Unix", "Darwin"):
    window = UnixWindow()
    button = UnixButton()
elif platform.system() == "Windows":
    window = WindowsWindow()
    button = WindowsButton()

button.set_color(Color.RED)
window.add_button(button)

In [29]:
if platform.system() in ("Unix", "Darwin"):
    window = NativeUnixWindow()
    button = NativeUnixButton()
    button.set_color("red")
    window.add_button(button)
elif platform.system() == "Windows":
    window = NativeWindowsWindow()
    button = NativeWindowsButton()
    button.color = (255, 0, 0)
    window.addButton(button)

## Плюсы

1. Сокращён спагетти код
2. Явный контракт для новых систем

## Минусы

1. Размер кода увеличился в х раз
2. Адаптер съел особенности

In [ ]:
# unix умеет мигать кнопкой
class NativeUnixButton(object):
    def set_color(self, color: str) -> None:
        pass
    
    def blink(self) -> None:
        pass

# windows не умеет мигать кнопкой
class NativeWindowButton(object):
    pass

In [ ]:
class UnixButton(ButtonAdapter):
    def blink(self) -> None:
        self._native_button.blink()

In [ ]:
class WindowsButton(ButtonAdapter):
    def blink(self) -> None:
        # ???

### Паттерн 2

In [ ]:
window = UnixWindow()
button = WindowsButton()
window.add_button(button)

# AbstractFactory
https://sourcemaking.com/design_patterns/abstract_factory

![abstract_factory](abstract_factory.png "AbstractFactory")

In [30]:
# /project/factory.py

import platform

# AbstractFactory
class PlatformFactory(ABC):
    @abstractmethod
    def create_window(self) -> WindowAdapter:
        pass

    @abstractmethod
    def create_button(self) -> ButtonAdapter:
        pass

In [31]:
class UnixFactory(PlatformFactory):
    def create_window(self) -> WindowAdapter:
        return UnixWindow()

    def create_button(self) -> ButtonAdapter:
        return UnixButton()


class WindowsFactory(PlatformFactory):
    def create_window(self) -> WindowAdapter:
        return WindowsWindow()

    def create_button(self) -> ButtonAdapter:
        return WindowsButton()

In [32]:
if platform.system() in ("Unix", "Darwin"):
    factory = UnixFactory()
elif platform.system() == "Windows":
    factory = WindowsFactory()

window = factory.create_window()
button = factory.create_button()
button.set_color(Color.RED)
window.add_button(button)

In [ ]:
if platform.system() in ("Unix", "Darwin"):
    window = NativeUnixWindow()
    button = NativeUnixButton()
    button.set_color("red")
    window.add_button(button)
elif platform.system() == "Windows":
    window = NativeWindowsWindow()
    button = NativeWindowsButton()
    button.color = (255, 0, 0)
    window.addButton(button)

## Плюсы

1. Убран спагетти код
2. Компоненты созданные фабрикой точно совместимы

## Минусы

1. Размера кода увеличился в х раз
2. Скованность создания объектов нового типа

In [ ]:
class NativeUnixButton(object):
    def __init__(self, title: str) -> None:
        self._title = title

In [ ]:
class UnixButton(ButtonAdapter):
    def __init__(self, title: str) -> None:
        self._native_button = NativeUnixButton(title)
        
class PlatformFactory(ABC):
    @abstractmethod
    def create_button(self, title: str) -> ButtonAdapter:
        pass
        
class UnixFactory(PlatformFactory):
    def create_button(self, title: str) -> ButtonAdapter:
        return UnixButton(title)
    
class NativeWindowsButton(object):
    def __init__(self) -> None:
        # No title

### Паттерн 3

In [33]:
# /project/interface.py
from enum import Enum


class Color(Enum):
    RED = 1
    GREEN = 2
    

class InteractiveElement(ABC):
    pass


class ButtonAdapter(InteractiveElement):
    @abstractmethod
    def set_color(self, color: Color) -> None:
        pass
    
    
class CheckboxAdapter(InteractiveElement):
    @abstractmethod
    def set_color(self, color: Color) -> None:
        pass
    
    
class WindowAdapter(ABC):
    @abstractmethod
    def add_element(self, element: InteractiveElement) -> None:
        pass

In [34]:
json_string = """
[
    {"type": "button", "color": "red"},
    {"type": "button", "color": "green"},
    {"type": "checkbox", "color": "red"}
]
"""

In [35]:
import json

config = json.loads(json_string)
print(config)

[{'type': 'button', 'color': 'red'}, {'type': 'button', 'color': 'green'}, {'type': 'checkbox', 'color': 'red'}]


In [ ]:
# /project/main.py
import json

def factory_for_system(system: tp.Optional[str]=None):
    system = system or platform.system()
    if platform.system() in ("Unix", "Darwin"):
        return UnixFactory()
    elif platform.system() == "Windows":
        return WindowsFactory()

factory = factory_for_system()
config = json.loads(json_string)
for element in config:
    color_str = element["color"]
    if color_str == "red":
        color = Color.RED
    elif color_str == "green":
        color = Color.GREEN
    else:
        raise ValueError("Wrong color value in config: '{}'".format(color_str))
    what = element["type"]
    if what == "button":
        element = factory.create_button()
    elif what == "checkbox":
        element = factory.create_checkbox()
    else:
        raise ValueError("Wrong element in config: '{}'".format(what))
    element.set_color(color)

# FactoryMethod
https://sourcemaking.com/design_patterns/factory_method

In [37]:
# /project/interface.py

class Color(Enum):
    RED = 1
    GREEN = 2
    
    @staticmethod
    def from_string(color_str: str) -> "Color":
        if color_str in ("red", "RED"):
            return Color.RED
        elif color_str in ("green", "GREEN"):
            return Color.GREEN
        else:
            raise ValueError("Wrong color value in config: '{}'".format(color_str))

In [38]:
Color.from_string("green")

<Color.GREEN: 2>

In [41]:
Color(1)

<Color.RED: 1>

In [ ]:
# /project/main.py
import json

factory = factory_for_system()
config = json.loads(json_string)
for element in config:
    color = Color.from_string(element["color"])
    what = element["type"]
    if what == "button":
        element = factory.create_button()
    elif what == "checkbox":
        element = factory.create_checkbox()
    element.set_color(color)

In [ ]:
class PlatformFactory(ABC):
    @abstractmethod
    def create_window(self) -> WindowAdapter:
        pass

    @abstractmethod
    def create_button(self) -> ButtonAdapter:
        pass
    
    @abstractmethod
    def create_checkbox(self) -> ButtonAdapter:
        pass
    
    # FactoryMethod inside AbstractFactory 
    def interactive_element_from_str(element_str: str) -> InteractiveElement:
        if element_str == "button":
            return self.create_button()
        elif element_str == "checkbox":
            return self.create_checkbox()
        else:
            raise ValueError("Unknown element type: '{}'".format(element_str))

In [ ]:
# /project/main.py
import json

from project.factory import factory_for_system

factory = factory_for_system()
config = json.loads(json_string)
for element in config:
    color = Color.from_string(element["color"])
    element = factory.interactive_element_from_str(element["type"])
    element.set_color(color)

## Плюсы


1. Убран спагетти код
2. Явные зоны ответственности

## Минусы

1. Размер кода увеличился в x раз
2. Скованность одним интерфейсом создания объектов

In [ ]:
# Новая опция hold
class ButtonAdapter(InteractiveElement):
    def __init__(self, hold: bool = False) -> None:
        self._hold = hold
    
    @abstractmethod
    def set_color(self, color: Color) -> None:
        pass
    
# Новая опция checked
class CheckboxAdapter(InteractiveElement):
    def __init__(self, checked: bool = False) -> None:
        self._checked = checked
    
    @abstractmethod
    def set_color(self, color: Color) -> None:
        pass

In [ ]:
# Обе опции тут
class PlatformFactory(ABC):
    def interactive_element_from_str(element_str: str, hold: bool, checked: bool) -> InteractiveElement:
        if element_str == "button":
            return self.create_button(hold)
        elif element_str == "checkbox":
            return self.create_checkbox(checked)
        else:
            raise ValueError("Unknown element type: '{}'".format(element_str))

### Сериалзовывать json в dict - плохой способ настраивать опции программы

## Дескрипторы

In [48]:
class Cow:
    def __init__(self, name: str) -> None:
        self._name = name
        
    # Проверяем имя коровы
    def set_name(self, name: str) -> None:
        if not isinstance(name, str):
            raise ValueError("Expected type <str> for name, got <{}>".format(type(name).__name__))
        if not name or not name.isalnum():
            raise ValueError("Name should be non-empty alphanumeric string")
        self._name = name
        
cow = Cow("Isabella")
for name in (1, "", "Ja Ra"):
    try:
        cow.set_name(name)
    except ValueError as e:
        print(e)

Expected type <str> for name, got <int>
Name should be non-empty alphanumeric string
Name should be non-empty alphanumeric string


In [ ]:
class Sheep:
    def __init__(self, name: str):
        self._name = name
        
    def set_name(self, name: str) -> None:
        if not isinstance(name, str):
            raise ValueError("Expected type <str> for name, got <{}>".format(type(name).__name__))
        if not name or not name.isalnum():
            raise ValueError("Name should be non-empty alphanumeric string")
        self._name = name

## Решения
### Наследование

In [49]:
class Animal:
    def __init__(self, name: str) -> None:
        self._name = name
        
    def set_name(self, name: str) -> None:
        if not isinstance(name, str):
            raise ValueError("Expected type <str> for name, got <{}>".format(type(name).__name__))
        if not name or not name.isalnum():
            raise ValueError("Name should be non-empty alphanumeric string")
        self._name = name
        
class Cow(Animal):
    pass

class Sheep(Animal):
    pass

cow = Cow("Isabella")
sheep = Sheep("Boris")
for animal in (cow, sheep):
    try:
        cow.set_name("")
    except ValueError as e:
        print(e)

Name should be non-empty alphanumeric string
Name should be non-empty alphanumeric string


### Проблема

In [ ]:
class Farmer:
    def __init__(self, name: str, surname: str) -> None:
        self._name = name
        self._surname = surname
        
    def set_name(self, name: str) -> None:
        if not isinstance(name, str):
            raise ValueError("Expected type <str> for name, got <{}>".format(type(name).__name__))
        if not name or name.isalnum():
            raise ValueError("Name should be non-empty alphanumeric string")
        self._name = name
        
    def set_surname(self, surname: str) -> None:
        if not isinstance(surname, str):
            raise ValueError("Expected type <str> for surname, got <{}>".format(type(surname).__name__))
        if not surname or not surname.isalnum():
            raise ValueError("Name should be non-empty alphanumeric string")
        self._surname = surname

In [ ]:
class Farmer(Animal):
    def __init__(self, name: str, surname: str) -> None:
        super().__init__()
        self._surname = surname
        
    def set_surname(self, surname: str) -> None:
        if not isinstance(surname, str):
            raise ValueError("Expected type <str> for surname, got <{}>".format(type(surname).__name__))
        if not surname or not surname.isalnum():
            raise ValueError("Name should be non-empty alphanumeric string")
        self._surname = surname

### Дескрипторы
https://docs.python.org/3/howto/descriptor.html

In [ ]:
# https://docs.python.org/3/howto/descriptor.html#overview-of-descriptor-invocation
b.x

In [ ]:
b.__getattribute__("x")
b.__getattribute__ == object.__getattribute__

In [ ]:
class object:
    def __getattribute__(obj: tp.Any, name: str) -> tp.Any:
        null = object()
        objtype = type(obj)
        cls_var = getattr(objtype, name, null)
        descr_get = getattr(type(cls_var), '__get__', null)
        if descr_get is not null:
            if (hasattr(type(cls_var), '__set__')
                or hasattr(type(cls_var), '__delete__')):
                return descr_get(cls_var, obj, objtype)     # data descriptor
        if hasattr(obj, '__dict__') and name in vars(obj):
            return vars(obj)[name]                          # instance variable
        if descr_get is not null:
            return descr_get(cls_var, obj, objtype)         # non-data descriptor
        if cls_var is not null:
            return cls_var                                  # class variable
        raise AttributeError(name)

In [ ]:
class Descriptor:
    def __get__(self, obj: tp.Optional[tp.Any], objtype: type) -> None:
        print(f"Descriptor.__get__(self={self}, obj={obj}, objtype={objtype})")


class Farmer:
    name = Descriptor()
    

farmer = Farmer()
farmer.name

In [ ]:
object.__getattribute__(farmer, "name") -> tp.Any:
    null = object()
    objtype = type(obj)  # Farmer
    cls_var = getattr(objtype, "name", null)  # Farmer.name
    descr_get = getattr(type(cls_var), '__get__', null)  # Descriptor.__get__
    if descr_get is not null:  # Descriptor.__get__ is not null
        if (hasattr(type(cls_var), '__set__')  # hasattr(Descriptor, '__set__')
            or hasattr(type(cls_var), '__delete__')):  # hasattr(Descriptor, '__delete__')
            return descr_get(cls_var, obj, objtype)  # Descriptor.__get__(Farmer.name, farmer, Farmer)
    if hasattr(farmer, '__dict__') and name in vars(obj):
        return vars(obj)[name]  # farmer.__dict__["name"]
    if Descriptor.__get__ is not null:
        return descr_get(cls_var, obj, objtype)  # Descriptor.__get__(Farmer.name, farmer, Farmer)
    if cls_var is not null:
        return cls_var  # Farmer.name
    raise AttributeError(name)

In [91]:
class Descriptor:
    def __get__(self, obj: tp.Optional[tp.Any], objtype: type) -> None:
        print(f"Descriptor.__get__(self={self}, obj={obj}, objtype={objtype})")


class Farmer:
    name = Descriptor()
    

farmer = Farmer()
farmer.name

Descriptor.__get__(self=<__main__.Descriptor object at 0x7fb955931278>, obj=<__main__.Farmer object at 0x7fb9559312b0>, objtype=<class '__main__.Farmer'>)


In [ ]:
Farmer.__dict__["name"].__get__(Farmer.__dict__["name"], farmer, Farmer)

In [ ]:
a = farmer.name
Farmer.name.__get__(farmer, Farmer)

farmer.name = 1
Farmer.name.__set__(farmer, 1)

del farmer.name
Farmer.name.__delete__(farmer, Farmer)

In [62]:
class NonEmptyString:
    """Descriptor for string members. Checks non empty string when set."""
    def __init__(self, name: str):
        self._name = name
    
    # print(farmer.name)
    def __get__(self, obj: tp.Optional[tp.Any], objtype: type) -> tp.Any:
        if obj is None:
            return self
        return getattr(obj, self._name)
    
    # farmer.name = "Kolya"
    def __set__(self, obj: tp.Any, value: str) -> None:
        if not isinstance(value, str):
            raise ValueError("Expected type <str> for value, got <{}>".format(type(value).__name__))
        if not value or not value.isalnum():
            raise ValueError("Value should be non-empty alphanumeric string")
        setattr(obj, self._name, value)
        
    # del farmer.name
    def __delete__(self, obj: tp.Any) -> None:
        raise ValueError("Value cannot be deleted")

In [65]:
class Farmer:
    name = NonEmptyString("_name")
    surname = NonEmptyString("_surname")
    
    def __init__(self, name: str, surname: str):
        self.name = name
        self.surname = surname
    

farmer = Farmer("Leo", "Pellegro")
print(farmer.name)
farmer.name = "Nick"
print(farmer.name)

Leo
Nick


In [66]:
another_farmer = Farmer("Jagienka", "Jankowski")
print(another_farmer.name, another_farmer.surname)
print(farmer.name, farmer.surname)

Jagienka Jankowski
Nick Pellegro


In [67]:
try:
    farmer.name = ""
except ValueError as e:
    print(e)

Value should be non-empty alphanumeric string


In [69]:
try:
    del farmer.name
except ValueError as e:
    print(e)

Value cannot be deleted


In [70]:
class BadFarmer:
    name = NonEmptyString("_name")
    surname = NonEmptyString("_surname")
    
    def __init__(self, name: str, surname: str) -> None:
        self.name = name

farmer = BadFarmer("Nikolai", "Jos")
try:
    print(farmer.surname)
except AttributeError as e:
    print(e)

'BadFarmer' object has no attribute '_surname'


In [134]:
# Python 3.6

class NonEmptyString2:
    """Descriptor for string members. Checks non empty string when set."""
    def __get__(self, obj: tp.Optional[tp.Any], objtype: type) -> tp.Any:
        if obj is None:
            return self
        return obj.__dict__[self._name]

    def __set__(self, obj: tp.Any, value: str) -> None:
        if not isinstance(value, str):
            raise ValueError("Expected type <str> for value, got <{}>".format(type(value).__name__))
        if not value or not value.isalnum():
            raise ValueError("Value should be non-empty alphanumeric string")
        obj.__dict__[self._name] = value
        
    def __delete__(self, obj: tp.Any) -> None:
        raise ValueError("Value cannot be deleted")

    def __set_name__(self, owner: tp.Any, name: str):
        self._name = name

In [137]:
class BadFarmer:
    name = NonEmptyString2()
    surname = NonEmptyString2()
    
    def __init__(self, name: str, surname: str) -> None:
        self.name = name

farmer = BadFarmer("Nikolai", "Jos")
try:
    print(farmer.surname)
except AttributeError as e:
    print(e)

KeyError: 'surname'

### Замечание 1: getattr_hook

In [ ]:
a = b.x

In [ ]:
a = getattr_hook(b, "x")

def getattr_hook(obj, name):
    "Emulate slot_tp_getattr_hook() in Objects/typeobject.c"
    try:
        return obj.__getattribute__(name)
    except AttributeError:
        if not hasattr(type(obj), '__getattr__'):
            raise
    return type(obj).__getattr__(obj, name)

### Замечание 2: __get__(obj=None)

In [92]:
class Descriptor:
    def __get__(self, obj: tp.Optional[tp.Any], objtype: type) -> None:
        print(f"Descriptor.__get__(self={self}, obj={obj}, objtype={objtype})")
        
class Farmer:
    name = Descriptor()

In [93]:
Farmer.name

Descriptor.__get__(self=<__main__.Descriptor object at 0x7fb955931240>, obj=None, objtype=<class '__main__.Farmer'>)


In [ ]:
Farmer.name != object.__getattribute__(Farmer, "name")
Farmer.name == type.__getattribute__(Farmer, "name")

### Замечание 3: non-data vs data descriptors

In [4]:
class Dog:
    @staticmethod
    def bark():
        print("bark")
    
d = Dog()
d.bark = lambda: print("meow")
d.bark()

meow


### Property

In [94]:
# https://docs.python.org/3/howto/descriptor.html#properties

class Animal:
    def __init__(self, usd_price: float) -> None:
        self._usd_price = usd_price
        
    def get_price(self) -> float:
        usd_to_rub = 67  # сходить за актуальным курсом usd
        return self._usd_price * usd_to_rub
    
    def set_price(self, price: float) -> None:
        if price <= 0:
            raise ValueError("Price must be positive, got {}".format(price))
        self._usd_price = price

In [95]:
animal = Animal(100)
print(animal.get_price())
animal.set_price(10)
print(animal.get_price())
try:
    animal.set_price(-10)
except ValueError as e:
    print(e)

6700
670
Price must be positive, got -10


In [96]:
class Animal:
    def __init__(self, usd_price):
        self._usd_price = usd_price
        
    @property
    def price(self):
        usd_to_rub = 67  # сходить за актуальным курсом usd
        return self._usd_price * usd_to_rub

In [97]:
animal = Animal(100)
print(animal.price)
animal.price = 10
print(animal.price)

6700


AttributeError: can't set attribute

In [98]:
class Animal:
    def __init__(self, usd_price):
        self._usd_price = usd_price
        
    @property
    def price(self):
        usd_to_rub = 67  # сходить за актуальным курсом usd
        return self._usd_price * usd_to_rub
    
    @price.setter
    def price(self, price):
        if price <= 0:
            raise ValueError("Price must be positive, got {}".format(price))
        self._usd_price = price

In [99]:
animal = Animal(100)
print(animal.price)
animal.price = 10
print(animal.price)
try:
    animal.price = -10
except ValueError as e:
    print(e)

6700
670
Price must be positive, got -10


In [ ]:
class Animal:
    def __init__(self, usd_price: float) -> None:
        self._usd_price = usd_price
        
    def get_price(self) -> float:
        usd_to_rub = 67  # сходить за актуальным курсом usd
        return self._usd_price * usd_to_rub
    
    def set_price(self, price: float) -> None:
        if price <= 0:
            raise ValueError("Price must be positive, got {}".format(price))
        self._usd_price = price
        
    price = property(get_price, set_price)

## Создание объектов

### Инициализация

In [ ]:
class Class1:      
    def __init__(self, x: int) -> None:
        print(f"Class1.__init__({x})")
        self.x = x
        
a = Class1(1)
print(type(a))
print(a.x)

In [100]:
class Class1:      
    def __init__(self, x: int) -> None:
        print(f"Class1.__init__({x})")
        self.x = x
        
a = Class1(1)
print(type(a))
print(a.x)

Class1.__init__(1)
<class '__main__.Class1'>
1


![__init__](meta/img/img.001.jpeg "__init__")

### Выделение памяти под объект

In [101]:
class Class:
    pass

In [102]:
c = Class()

In [103]:
import dis

def f():
    return Class()
    
dis.dis(f)

  4           0 LOAD_GLOBAL              0 (Class)
              2 CALL_FUNCTION            0
              4 RETURN_VALUE


In [ ]:
# kindof
def Class(*args, **kwargs) -> Class:
    ???

In [ ]:
# kindof
def Class(*args, **kwargs) -> Class:
    # выделить место под объект
    # инициализировать объект
    # вернуть готовый объект

In [ ]:
# kindof
def Class(*args, **kwargs) -> Class:
    # выделить место под объект
    obj = ???
    # инициализировать объект
    Class.__init__(obj, *args, **kwargs)
    # вернуть готовый объект
    return obj

In [ ]:
# kindof
def Class(*args, **kwargs) -> Class:
    # выделить место под объект
    obj = Class.__new__(*args, **kwargs)
    # инициализировать объект
    Class.__init__(obj, *args, **kwargs)
    # вернуть готовый объект
    return obj

In [ ]:
class Class2:
    def __new__(cls, *args) -> "Class2":
        print(f"Class2.__new__(cls={cls}, args={args})")
        return object.__new__(cls)
    
    def __init__(self, *args) -> None:
        print(f"Class2.__init__({args})")
        
c = Class2(2)
print(type(c))

In [104]:
class Class2:
    def __new__(cls, *args) -> "Class2":
        print(f"Class2.__new__(cls={cls}, args={args})")
        return object.__new__(cls)
    
    def __init__(self, *args) -> None:
        print(f"Class2.__init__({args})")
        
c = Class2(2)
print(type(c))

Class2.__new__(cls=<class '__main__.Class2'>, args=(2,))
Class2.__init__((2,))
<class '__main__.Class2'>


In [105]:
c = Class2(1)
print("-")
c = Class2.__new__(Class2, 1)
c.__init__(1)

Class2.__new__(cls=<class '__main__.Class2'>, args=(1,))
Class2.__init__((1,))
-
Class2.__new__(cls=<class '__main__.Class2'>, args=(1,))
Class2.__init__((1,))


In [ ]:
# Забавная штука
class Class2:
    def __new__(cls, *args) -> int:
        return 1
    
    def __init__(self, *args) -> None:
        self.x = args[0]
        
x = Class2()
print(type(x))
print(x.x)

In [106]:
# Забавная штука
class Class2:
    def __new__(cls, *args) -> int:
        return 1
    
    def __init__(self, *args) -> None:
        self.x = args[0]
        
x = Class2()
print(type(x))
print(x.x)

<class 'int'>


AttributeError: 'int' object has no attribute 'x'

In [ ]:
c = Class2(1)
print("-")
c = Class2.__new__(Class2, 1)
if isinstance(c, Class2):
    c.__init__(1)

![__new__](meta/img/img.002.jpeg "__new__")

### Загадка

In [ ]:
class Class:
    pass

In [107]:
x = Class()

In [108]:
def func():
    pass

print(Class.__dict__)
print(func.__dict__)

{'__module__': '__main__', '__dict__': <attribute '__dict__' of 'Class' objects>, '__weakref__': <attribute '__weakref__' of 'Class' objects>, '__doc__': None}
{}


In [109]:
class Class:
    @classmethod
    def __call__(cls):
        raise RuntimeError()

x = Class()
x

In [110]:
x()

RuntimeError: 

In [ ]:
# https://docs.python.org/3/library/functions.html#type

class X:
    a = 1

X = type("X", (object, ), dict(a=1))

In [111]:
Class = type("Class", (object, ), dict())
print(Class.__call__)
print(Class.__call__())

<method-wrapper '__call__' of type object at 0x7fb94f695858>


In [1]:
def class_call(self):
    raise RuntimeError()

Class = type("Class", (object, ), {"__call__": class_call})
x = Class()
x()

RuntimeError: 

### PyTypeObject

```c++
struct PyTypeObject {
    PyObject* tp_name;
    PyObject* tp_new;
    PyObject* tp_init;
    PyObject* tp_call;
    PyObject* tp_dict;
};
```

In [ ]:
class X:
    def __new__(...):
        pass
    
    def __init__(...):
        pass
    
    def __call__(...):
        pass

```c++
PyTypeObject PyType_X;
PyType_X.tp_name = "X";
PyType_X.tp_new = __new__
PyType_X.tp_init = __init__
PyType_X.tp_call = type_call
PyType_X.tp_dict = {"__call__": __call__}
```

In [1]:
from IPython.display import HTML
HTML('''<style>.CodeMirror{min-width:100% !important;}</style>''')

```c++
// Objects/typeobject.c
// very distilled
static PyObject *
type_call(PyTypeObject *type, PyObject *args, PyObject *kwds)
{
    PyObject *obj;

    if (type->tp_new == NULL) {
        // assert
    }

    obj = type->tp_new(type, args, kwds);
    if (obj == NULL)
        return NULL;
    
    if (!PyType_IsSubtype(Py_TYPE(obj), type))
        return obj;

    if (type->tp_init != NULL) {
        int res = type->tp_init(obj, args, kwds);
        // assert if res < 0
    }
    return obj;
}
```

### Почему X() всегда работает?

In [ ]:
X()

```c++
X.tp_call()
// X.tp_call == type_call
```

### Почему X.\_\_call__ показывает непонятно что?

In [ ]:
X.__call__
X.__getattribute__("__call__")

In [ ]:
def __getattribute__(cls, name):
    found = getattr(cls, name)
    if isdescriptor(found):
        return found
    if name in cls.__dict__:
        return cls.__dict__[name]:
    return found

```c++
// Objects/typeobject.c
// getattr()
static PyObject *
type_getattro(PyTypeObject *type, PyObject *naae)
{
    PyTypeObject *metatype = Py_TYPE(type);
    PyObject *meta_attribute, *attribute;
    descrgetfunc meta_get;
    PyObject* res;

    if (!PyUnicode_Check(name)) {
        PyErr_Format(PyExc_TypeError,
                     "attribute name must be string, not '%.200s'",
                     name->ob_type->tp_name);
        return NULL;
    }

    /* Initialize this type (we'll assume the metatype is initialized) */
    if (type->tp_dict == NULL) {
        if (PyType_Ready(type) < 0)
            return NULL;
    }

    /* No readable descriptor found yet */
    meta_get = NULL;

    /* Look for the attribute in the metatype */
    meta_attribute = _PyType_Lookup(metatype, name);

    if (meta_attribute != NULL) {
        Py_INCREF(meta_attribute);
        meta_get = Py_TYPE(meta_attribute)->tp_descr_get;

        if (meta_get != NULL && PyDescr_IsData(meta_attribute)) {
            /* Data descriptors implement tp_descr_set to intercept
             * writes. Assume the attribute is not overridden in
             * type's tp_dict (and bases): call the descriptor now.
             */
            res = meta_get(meta_attribute, (PyObject *)type,
                           (PyObject *)metatype);
            Py_DECREF(meta_attribute);
            return res;
        }
    }

    /* No data descriptor found on metatype. Look in tp_dict of this
     * type and its bases */
    attribute = _PyType_Lookup(type, name);
    if (attribute != NULL) {
        /* Implement descriptor functionality, if any */
        Py_INCREF(attribute);
        descrgetfunc local_get = Py_TYPE(attribute)->tp_descr_get;

        Py_XDECREF(meta_attribute);

        if (local_get != NULL) {
            /* NULL 2nd argument indicates the descriptor was
             * found on the target object itself (or a base)  */
            res = local_get(attribute, (PyObject *)NULL,
                            (PyObject *)type);
            Py_DECREF(attribute);
            return res;
        }

        return attribute;
    }

    /* No attribute found in local __dict__ (or bases): use the
     * descriptor from the metatype, if any */
    if (meta_get != NULL) {
        PyObject *res;
        res = meta_get(meta_attribute, (PyObject *)type,
                       (PyObject *)metatype);
        Py_DECREF(meta_attribute);
        return res;
    }

    /* If an ordinary attribute was found on the metatype, return it now */
    if (meta_attribute != NULL) {
        return meta_attribute;
    }

    /* Give up */
    PyErr_Format(PyExc_AttributeError,
                 "type object '%.50s' has no attribute '%U'",
                 type->tp_name, name);
    return NULL;
}
```

### Итого

__Python__
```python
class X:
    a = 1
    
    def __new__(...):
        pass
    
    def __init__(...):
        pass
```

__C__
```c++
PyTypeObject PyType_X;
PyType_X.tp_name = "X";
PyType_X.tp_new = __new__
PyType_X.tp_init = __init__
PyType_X.tp_call = type_call
PyType_X.tp_dict = {"a": 1}


PyObject* type_call() {
    PyObject* x = PyType_X.tp_new();
    PyType_X.tp_init(x);
    return x
}
```

### metaclass.\_\_call__

In [ ]:
class Class3:
    def __new__(cls, *args, **kwargs) -> "Class3":
        print(f"Class3.__new__({cls}, {args}, {kwargs})")
        return object.__new__(cls)
    
    def __init__(
        self, 
        *args, 
        **kwargs
    ) -> None:
        print(f"Class3.__init__({self}, {args}, {kwargs})")

In [126]:
class MetaclassA(type):
    def __call__(cls, *args, **kwargs) -> "Class3":
        print(f"MetaclassA.__call__({cls}, {args}, {kwargs})")
        obj = cls.__new__(cls, *args, **kwargs)
        obj.__init__(*args, **kwargs)
        return obj
        
class Class3(metaclass=MetaclassA):
    def __new__(cls, *args, **kwargs) -> "Class3":
        print(f"Class3.__new__({cls}, {args}, {kwargs})")
        return object.__new__(cls)
    
    def __init__(self, *args, **kwargs) -> None:
        print(f"Class3.__init__({self}, {args}, {kwargs})") 
        
x = Class3(1)
print(x)

MetaclassA.__call__(<class '__main__.Class3'>, (1,), {})
Class3.__new__(<class '__main__.Class3'>, (1,), {})
Class3.__init__(<__main__.Class3 object at 0x7fb955c413c8>, (1,), {})


In [ ]:
# Class3 = type("Class3", (object, ), dict())
Class3 = MetaclassA("Class3", (object, ), dict())

In [ ]:
Class3()

```c++
PyType_Class3.tp_call != type_call
PyType_Class3.tp_call == MetaclassA.__call__
type_call == type.tp_call
```

![__call__](meta/img/img.003.jpeg "__call__")

```c++
PyObject *
_PyObject_New(PyTypeObject *tp)
{
    PyObject *op = (PyObject *) PyObject_MALLOC(_PyObject_SIZE(tp));
    if (op == NULL) {
        return PyErr_NoMemory();
    }
    _PyObject_Init(op, tp);
    return op;
}

static inline void
_PyObject_Init(PyObject *op, PyTypeObject *typeobj)
{
    assert(op != NULL);
    Py_SET_TYPE(op, typeobj);
    if (_PyType_HasFeature(typeobj, Py_TPFLAGS_HEAPTYPE)) {
        Py_INCREF(typeobj);
    }
    _Py_NewReference(op);
}
```

### Flyweight

https://sourcemaking.com/design_patterns/flyweight

In [5]:
class Image:
    def __init__(self, path: str):
        self._path = path
    
    def __repr__(self):
        return "Image(path={})".format(self._path)

In [ ]:
class Image:
    def __init__(self, path: str):
        # load image to ram
        # slow and ram consuming

        
class Tree:
    def __init__(self, x: int, y: int, path: str):
        self._x = x
        self._y = y
        self._image = Image(path)

In [ ]:
def create_forest() -> tp.List[Tree]:
    trees = []
    for i in range(100):
        path = "imgs/tree{}.png".format(i % 3)
        trees.append(Tree(x, 10, path))

In [ ]:
class Tree:
    def __init__(self, x: int, y: int, path: tp.Optional[str] = None, image: tp.Optional[Image] = None):
        if image is not None:
            self._image = image
        elif path is not None:
            self._image = Image(path)
        else:
            raise RuntimeError("Either path or image must be specified")
        self._x = x
        self._y = y
        
image_lib = ImageLibrary()
tree = Tree(x, y, image=ImageLibrary.get("imgs/tree.png"))

In [12]:
class Flyweight(type):
    IMAGES = {}
    
    def __call__(cls, x: int, y: int, path: tp.Optional[str] = None, image: tp.Optional[Image] = None) -> "Tree":
        obj = cls.__new__(cls)
        if path is not None:
            if path in Flyweight.IMAGES:
                image = Flyweight.IMAGES[path]
            else:
                image = Image(path)
                Flyweight.IMAGES[path] = image
        cls.__init__(obj, x, y, path=path, image=image)
        return obj
    
class Tree(metaclass=Flyweight):
    def __init__(self, x: int, y: int, path: tp.Optional[str] = None, image: tp.Optional[Image] = None):
        if image is not None:
            self._image = image
        elif path is not None:
            self._image = Image(path)
        else:
            raise RuntimeError("Either path or image must be specified")
        self._x = x
        self._y = y

In [13]:
tree1 = Tree(1, 2, "tree.png")
tree2 = Tree(2, 3, "tree.png")
assert tree1._image is tree2._image

### type()

In [ ]:
class X:
    a = 1
    
X = type("X", (object, ), dict(a=1))

In [ ]:
class type:
    @classmethod
    def __call__(...):
        # smth

X = type.__call__("X", (object, ), dict(a=1))

In [ ]:
class type:
    @classmethod
    def __call__(...):
        # smth
        
type = type("type", (object, ), {"__call__": ...})

```c++
PyTypeObject PyType_Type;
PyType_Type.tp_name = "type";
PyType_Type.tp_new = type_new
PyType_Type.tp_init = type_init
PyType_Type.tp_call = type_call
```

In [ ]:
class Class:
    def __init__(self, x):
        self.x = x

obj = Class(1)

In [ ]:
obj = Class.__new__(Class, 1)
Class.__init__(obj, 1)

In [ ]:
X = type("X", (object, ), dict(a=1))

X = type.__new__(type, "X", (object, ), dict(a=1))
type.__init__(X, "X", (object, ), dict(a=1))

type.\_\_new__ возвращает не объект type, а наследник объекта type

In [ ]:
class MetaclassB(type):
    def __new__(
        cls: type,
        name: str,
        bases: tp.Tuple[type],
        members: tp.Dict[str, tp.Any]
    ):
        print(f"MetaclassB.__new__(cls={cls}, name={name}, bases={bases}, members={members})")
        return type.__new__(cls, name, bases, members)
        
    def __init__(
        cls: type,
        name: str, 
        bases: tp.Tuple[type], 
        members: tp.Dict[str, tp.Any]
    ):
        print(f"MetaclassB.__init__(cls={cls}, name={name}, bases={bases}, members={members})")
        
class Class4(metaclass=MetaclassB):
    pass

In [138]:
class MetaclassB(type):
    def __new__(
        cls: type,
        name: str,
        bases: tp.Tuple[type],
        members: tp.Dict[str, tp.Any]
    ):
        print(f"MetaclassB.__new__(cls={cls}, name={name}, bases={bases}, members={members})")
        return type.__new__(cls, name, bases, members)
        
    def __init__(
        cls: type,
        name: str, 
        bases: tp.Tuple[type], 
        members: tp.Dict[str, tp.Any]
    ):
        print(f"MetaclassB.__init__(cls={cls}, name={name}, bases={bases}, members={members})")
        
class Class4(metaclass=MetaclassB):
    pass

MetaclassB.__new__(cls=<class '__main__.MetaclassB'>, name=Class4, bases=(), members={'__module__': '__main__', '__qualname__': 'Class4'})
MetaclassB.__init__(cls=<class '__main__.Class4'>, name=Class4, bases=(), members={'__module__': '__main__', '__qualname__': 'Class4'})


### финалочка

In [ ]:
class Metaclass(type):
    pass

class Class(metaclass=Metaclass):
    pass

c = Class()

In [ ]:
Class = Metaclass.__new__(Metaclass, "Class", (object, ), dict())
Metaclass.__init__(Class)

c = Class.__new__(Class)
Class.__init__(c)

![all](meta/img/img.004.jpeg "all")

## ABC + abstractmethod

## Настоящий ABC скорее всего сложнее, и нижеследующий наверное не работает в каких-то случаях

__План__
1. Заменить декоратором `@abstractmethod` функции на объекты AbstractMethod
2. В момент создания класса собрать все объекты AbstractMethod в свойство `__abstractmethods__`
3. В момент создания объекта класса проверять пустоту `__abstractmethods__`

In [6]:
import typing as tp

In [7]:
class AbstractMethod:
    def __call__(self) -> None:
        raise NotImplementedError("Method not implemented")

def abstractmethod(method: tp.Callable[..., tp.Any]) -> AbstractMethod:
    return AbstractMethod()

class Animal():
    @abstractmethod
    def hello(self) -> None:
        pass
    
l = Animal()
try:
    l.hello()
except NotImplementedError as e:
    print(e)

Method not implemented


In [8]:
from copy import deepcopy

import inspect


class MyABCMeta(type):
    def __init__(
        cls: type, 
        name: str, 
        bases: tp.Tuple[type], 
        dct: tp.Dict[str, tp.Any]
    ) -> None:
        # Собираем все AbstractMethod из класса, который создаём
        abstract_methods = {name for name, value in dct.items() if isinstance(value, AbstractMethod)}
        # Собираем все AbstractMethod из родителей класса, который создаём
        for base in bases:
            new_methods = inspect.getmembers(base, predicate=lambda x: isinstance(x, AbstractMethod))
            abstract_methods.update({k for k, v in new_methods})
        # Теперь в abstract_methods собрали все методы, которые нужно переписать
        # Собираем все функции, которые есть в классе, который создаём
        concrete_methods = {name for name, value in dct.items() if inspect.isfunction(value)}
        # Записываем все непереопределённые методы в __abstract_methods__
        cls.__abstract_methods__ = abstract_methods - concrete_methods
        
    def __call__(
        cls: type, 
        *args: tp.Any,
        **kwargs: tp.Any
    ) -> tp.Any:
        # Если на момент создания объекта в классе остаются абстрактные методы кидаем ошибку
        if cls.__abstract_methods__:
            methods = ", ".join(cls.__abstract_methods__)
            raise NotImplementedError("Methods not implemented: {}".format(methods))
        return type.__call__(cls, *args, **kwargs)

class MyABC(metaclass=MyABCMeta):
   pass

In [9]:
class Animal(MyABC):
    @abstractmethod
    def hello(self) -> None:
        pass

    
class Cow(Animal):
    def hello(self) -> None:
        print("Moo")
        
class Sheep(Animal):
    pass

In [10]:
try:
    l = Animal()
except NotImplementedError as e:
    print(e)
try:
    s = Sheep()
except NotImplementedError as e:
    print(e)

c = Cow()
c.hello()

Methods not implemented: hello
Methods not implemented: hello
Moo


## Спасибо
## Вопросы?